In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import time
import datetime

1) Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).


In [30]:
data = pd.read_csv('features.csv', index_col='match_id')
X = data.iloc[:, :102]
y = data['radiant_win']

data_test = pd.read_csv('features_test.csv', index_col='match_id')
X_test = data_test

2) Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

3) Замените пропуски на нули с помощью функции fillna().

In [31]:
mass_prop = []
for i in range(102):
    if X.count()[i] != 97230:
        mass_prop.append(X.iloc[:,i].name)

X = X.fillna(0)
X_test = X_test.fillna(0)

In [32]:
display("Название признаков, имеющих пропуски", mass_prop)

'Название признаков, имеющих пропуски'

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

###### Например, признак 'first_blood_time', что означает время первого убийства в игре, может отсутствовать по причине того, что за первые 5 минут игры ни один из игроков не умер, следовательно это признак пуст. Признак 'radiant_courier_time', что означает время поупки курьера, также может отсутствовать по причине того, что команда 'Radient' не купила курьера за первые 5 минут игры.

4) Какой столбец содержит целевую переменную? Запишите его название.

In [33]:
display(y)

match_id
0         1
1         1
2         0
3         0
4         0
         ..
114402    0
114403    1
114404    0
114405    0
114406    1
Name: radiant_win, Length: 97230, dtype: int64

###### Столбец 'radiant_win' содержит целевую переменную

### Подход 1: градиентный бустинг "в лоб"

5) Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [35]:
n_estimators = [10, 20, 30, 40, 50]
mass = []
times = []

scaler = StandardScaler()
X = scaler.fit_transform(X)

for i in n_estimators:
    start_time = datetime.datetime.now()
    
    kf = KFold(n_splits=5, shuffle=True, random_state=241)
    gb = GradientBoostingClassifier(n_estimators=i, random_state=241, max_depth=2)
    gb.fit(X, y)
    scores = cross_val_score(gb, X, y, cv=kf, scoring='roc_auc')
    mass.append(scores)
    
    times.append(datetime.datetime.now() - start_time)

In [36]:
print(mass)
print(times)

[array([0.65579464, 0.64187792, 0.65037059, 0.65524734, 0.65092211]), array([0.67890027, 0.66540691, 0.67360983, 0.67477584, 0.67495895]), array([0.68925707, 0.67549772, 0.68101969, 0.68375468, 0.68581689]), array([0.69452829, 0.68145108, 0.68611001, 0.68773872, 0.6897242 ]), array([0.69832237, 0.68587487, 0.68975063, 0.69070186, 0.69310454])]
[datetime.timedelta(seconds=38, microseconds=840994), datetime.timedelta(seconds=71, microseconds=908493), datetime.timedelta(seconds=103, microseconds=277146), datetime.timedelta(seconds=131, microseconds=398079), datetime.timedelta(seconds=160, microseconds=267934)]


    Кросс-валидация для градиентного бустинга с 30 деревьями имеет время настройки 103 секунды
    
    Лучшее качество градиентного бустинга достигнуто, примерно 0.685 при n_estimators = 30 и  0.69 при n_estimators = 40. Классификатор настраивался долго. Как видно, достингнуто оптимальное качетвоа так как, качетво не должно расти при росте числа деревьев, либо растет, но незначительно. следовательно нет смысл использовать больше 30 деревьев в градиентном бустинге.
    
    Чтобы ускорить обучение при увеличении количества деревьев, я упрощал модель, использую max_depth = 2

### Подход 2: логистическая регрессия

1) Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

In [37]:
C = [10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3]
mass = []
scaler = StandardScaler()
X = scaler.fit_transform(X)

for i in C:
    kf = KFold(n_splits=5, shuffle=True, random_state=241)
    lg = LogisticRegression(penalty='l2', random_state=241, C=i, solver='lbfgs')
    lg.fit(X, y)
    scores = cross_val_score(lg, X, y, cv=kf, scoring='roc_auc')
    mass.append(scores)

In [38]:
mass

[array([0.72134691, 0.71368543, 0.7142503 , 0.71511826, 0.716496  ]),
 array([0.72166336, 0.71377455, 0.7144095 , 0.7154027 , 0.71646035]),
 array([0.72165999, 0.71370262, 0.71439004, 0.71539378, 0.71640597]),
 array([0.72165968, 0.71369426, 0.71438569, 0.71539326, 0.71640028]),
 array([0.72165847, 0.71369387, 0.71438513, 0.71539504, 0.71639887]),
 array([0.72165854, 0.71369377, 0.71438504, 0.7153948 , 0.71640154]),
 array([0.72165852, 0.7136937 , 0.71438502, 0.71539475, 0.7164022 ])]

Лучший параметр регуляризации C = 0.01. Наилучшее качество  0.72. Логистическая регрессия работает быстрее градиентного бустинга, и лучше по качеству.


2) Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [39]:
data = pd.read_csv('features.csv', index_col='match_id')
X = data.iloc[:, :102]
y = data['radiant_win']

data_test = pd.read_csv('features_test.csv', index_col='match_id')
X_test = data_test

X = X.fillna(0)
X_test = X_test.fillna(0)

del X['lobby_type']
for i in range(1,6):
    del X['r'+str(i)+'_hero']
    del X['d'+str(i)+'_hero']

In [40]:
mass = []
scaler = StandardScaler()
X = scaler.fit_transform(X)

grid = {'C': np.power(10.0, np.arange(-5,6))}
kf = KFold(n_splits=5, shuffle=True, random_state=241)
lg = LogisticRegression(penalty='l2', random_state=241, solver='lbfgs')
gs = GridSearchCV(lg, grid, scoring='roc_auc', cv=kf)
gs.fit(X, y)

scores = cross_val_score(lg, X, y, cv=kf, scoring='roc_auc')
mass.append(scores)

In [42]:
gs.cv_results_

{'mean_fit_time': array([0.45552368, 0.50249729, 0.82209506, 1.18991156, 1.19189386,
        1.25988736, 1.23083825, 1.25682731, 1.24284654, 1.30579638,
        1.25587482]),
 'std_fit_time': array([0.0272085 , 0.01554925, 0.02845215, 0.08291699, 0.09083502,
        0.04590073, 0.03661275, 0.07078653, 0.07645949, 0.05034263,
        0.06168946]),
 'mean_score_time': array([0.01538229, 0.0145854 , 0.01338758, 0.01621981, 0.01602063,
        0.014569  , 0.01499014, 0.01518559, 0.01377444, 0.01618509,
        0.01598663]),
 'std_score_time': array([0.00162398, 0.00149683, 0.00049171, 0.002279  , 0.00259012,
        0.00212546, 0.00167189, 0.00213584, 0.00098422, 0.00271266,
        0.00379193]),
 'param_C': masked_array(data=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                    1000.0, 10000.0, 100000.0],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False],
        fill_value='?',
             dtype=object)

###### Качество практичечки не изменилось, это можно объяснить тем, что модель не учитывала категориальные признаки, а сейчас мы их удалили и убелись в этом.

3) На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [74]:
data = pd.read_csv('features.csv', index_col='match_id')
X = data.iloc[:, :102]
y = data['radiant_win']

data_test = pd.read_csv('features_test.csv', index_col='match_id')
X_test = data_test

X = X.fillna(0)
X_test = X_test.fillna(0)

print(sorted(X['d1_hero'].unique()))

for i in range(1,6):
    print(len(X['r'+str(i)+'_hero'].unique()))
    print(len(X['d'+str(i)+'_hero'].unique()))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 109, 110, 112]
108
108
108
108
108
108
108
108
108
108


###### Как можно видеть из данных, существует всего 108 различных идентификтаоров героев. Последний идентификатор заканчивается на 112

4) Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [75]:
N = 112
X_pick = np.zeros((X.shape[0], N))

for i, match_id in enumerate(X.index):
    for p in range(5):
        X_pick[i, X.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
X_pick = pd.DataFrame(X_pick)
del X['lobby_type']
del X_test['lobby_type']
for i in range(1,6):
    del X['r'+str(i)+'_hero']
    del X['d'+str(i)+'_hero']
    del X_test['r'+str(i)+'_hero']
    del X_test['d'+str(i)+'_hero']

X.index = np.arange(97230)
X_test.index = np.arange(17177)

X = X.join(X_pick)

#X = X.iloc[:10000]

X_test = X_test.join(X_pick)

#X_test = X_test.iloc[:10000]
#y = y[:10000]

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,102,103,104,105,106,107,108,109,110,111
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97225,1450265551,4,1706,1198,17,0,1,8,2,616,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97226,1450277704,4,1793,1416,17,0,1,5,3,764,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
97227,1450291848,4,1399,540,1,0,0,5,4,1448,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
97228,1450292986,3,1135,766,6,0,2,6,5,1954,...,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0


6) Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [76]:
mass = []
scaler = StandardScaler()
X = scaler.fit_transform(X)

grid = {'C': np.power(10.0, np.arange(-5,6))}
kf = KFold(n_splits=5, shuffle=True, random_state=241)
lg = LogisticRegression(penalty='l2', random_state=241, solver='lbfgs')
gs = GridSearchCV(lg, grid, scoring='roc_auc', cv=kf)
gs.fit(X, y)

scores = cross_val_score(lg, X, y, cv=kf, scoring='roc_auc')
mass.append(scores)

###### Полученное качество
Качество улучшилось, так как мы правильно описали категориальные признаки

In [77]:
mass

[array([0.75512029, 0.74989619, 0.75212706, 0.74953815, 0.75290589])]

In [78]:
gs.cv_results_

{'mean_fit_time': array([0.89984779, 1.07913575, 1.72093873, 2.47166958, 3.43426757,
        2.48386412, 2.53861399, 2.47802267, 2.49342709, 2.48935318,
        2.484589  ]),
 'std_fit_time': array([0.03332061, 0.10885725, 0.12274122, 0.0923274 , 0.60673411,
        0.01895641, 0.05241989, 0.04183304, 0.02396092, 0.02886847,
        0.02436187]),
 'mean_score_time': array([0.02178078, 0.02018218, 0.02118206, 0.02558246, 0.05596409,
        0.01878481, 0.01904511, 0.02018704, 0.01775684, 0.01980748,
        0.02017937]),
 'std_score_time': array([4.07046575e-03, 3.36916561e-03, 2.31468735e-03, 5.67466666e-03,
        6.94934370e-02, 3.99865940e-04, 9.37536432e-04, 3.26100841e-05,
        3.91131789e-03, 8.13125823e-04, 3.51042839e-05]),
 'param_C': masked_array(data=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0,
                    1000.0, 10000.0, 100000.0],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False],
      

Как видно, лучший результат получается при С = 0.01

###### Можно сделать вывод, что из  логистической регрессии получается более верная модель (0.75 > 0.69). Сделаем предсказания

In [79]:
X_test = scaler.transform(X_test)
pred = gs.predict_proba(X_test)[:, 1]
print('Минимальное значение: ', pred.min())
print('Максимальное значение: ', pred.max())

Минимальное значение:  0.00472219927337751
Максимальное значение:  0.9974818227361194
